# Spatial Data Analysis and Econometrics with PySAL
## Spatial Regression Module - spreg

### Luc Anselin and Pedro Amaral

### (revised 10/30/2024)


## Preliminaries

In this notebook, we present a brief overview of the main functionalities of the `spreg` library in PySAL. `spreg` (short for spatial regression) supports the estimation of classic and spatial regression models. Currently, it contains methods for estimating standard and spatial versions of models such as Ordinary Least Squares (OLS), Two Stage Least Squares (2SLS), Seemingly Unrelated Regressions (SUR), and Random and Fixed effects panels. Additionally, it offers various tests of homoskedasticity, normality, spatial randomness, different types of spatial autocorrelation, and tools for the generation of random spatial data and model specification search. It addresses spatial heterogeneity through spatial regimes regression with both exogenous and endogenous determination of the regimes.

In this workshop, we will focus exclusively on spatial dependence and cover the following:
1. Basic Data Management
2. OLS with Spatial Diagnostics
3. Specification Search
4. The SLX model
5. Spatial Lag and Spatial Durbin models 
6. Spatial Impacts

This notebook is adapted from Luc Anselin's course material available in much more detail at https://github.com/lanselin/spatial_regression_notebooks and on the GeoDa Center YouTube channel, at https://www.youtube.com/@GeoDaCenter/playlists.

### Prerequisites

Familiarity with the spatial regression specifications and the basic estimation methods is assumed. Very little is required in terms of software prerequisites. Sample data files are examined and loaded with *libpysal*; *geopandas* and *pandas* are used to read the data, and *spreg* for all estimation and diagnostics. 

## 1. Basic Data Management


This section covers some elementary functionality to carry out data input. The key concept is a so-called *DataFrame*, a tabular representation of the data with observations as rows and variables as columns.

### Modules Needed

The three modules needed to work with sample data are *libpysal*, *pandas* and *geopandas*. 

Some additional imports are included to avoid excessive warning messages. With later versions of PySAL, these may not be needed.

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['USE_PYGEOS'] = '0'

import pandas as pd
import geopandas as gpd
import libpysal

### Functionality Used

- from pandas/geopandas:
  - read_file
  
- from libpysal:
  - examples.get_path

### Input Files

The material in this notebook assumes that the **chicagoSDOH** PySAL sample data set has been loaded (e.g., by means of `libpysal.examples.load_example("chicagoSDOH")`). All data are extracted from the files included in this sample data set.

The input file is specified generically as **infileshp** (for the **Chi-SDOH** shape file). 

In [2]:
infileshp = "Chi-SDOH.shp"            # input shape file with data

### Reading the Input File from the Example Data Set

The actual path to the files contained in the local copy of the remote data set is found by means of `libpysal.examples.get_path`. This is then passed to the *geopandas* `read_file` function in the usual way. We list the dimensions of the data set (number of observations and number of variables) as well as the complete set of variable names (`columns`).

In [3]:
inpath = libpysal.examples.get_path(infileshp)
dfs = gpd.read_file(inpath)
print(dfs.shape)
print(dfs.columns)

(791, 67)
Index(['ID', 'OBJECTID', 'TRACTCE10', 'geoid10', 'commarea', 'Shape_Leng',
       'Shape_Area', 'PDENS14', 'CarC14P', 'CAR', 'NOCAR', 'CTA14P', 'CTA',
       'CmTm14', 'Undr514P', 'Wht14P', 'WHT50PCT', 'Wht', 'Blk14P',
       'BLCK50PCT', 'Blk', 'Hisp14P', 'HISP50PCT', 'Hisp', 'Pop2014',
       'MEANMI_14', 'FACHANGE', 'PCRIMERT15', 'VCRIMERT15', 'COI_ct', 'HIS_ct',
       'YEARS_LOST', 'YPLL_rate', 'ForclRt', 'EP_PCI', 'EP_MUNIT', 'EP_GROUPQ',
       'SSWS2USE', 'SchHP_Mi', 'BrownF_Mi', 'MEANMI_07', 'MEANMI_11',
       'EP_MINRTY', 'Ovr6514P', 'EP_AGE17', 'EP_DISABL', 'EP_NOHSDP',
       'EP_LIMENG', 'EP_SNGPNT', 'Pov14', 'PerCap14', 'Unemp14', 'EP_UNINSUR',
       'EP_CROWD', 'EP_NOVEH', 'ChldPvt14', 'HealthLit', 'FORCLRISK',
       'COORD_X', 'COORD_Y', 'C_X', 'C_Y', 'districtno', 'district', 'region',
       'regionno', 'geometry'],
      dtype='object')


## 2. OLS with Specification Tests

In this section, the basic OLS estimation and elementary non-spatial regression diagnostics are reviewed. In addition, diagnostics for spatial autocorrelation are covered as well.

Technical details are treated in Chapter 5 in Anselin and Rey (2014). *Modern Spatial Econometrics in Practice*.

### Modules Needed

The main module for spatial regression in PySAL is *spreg*. The examples in this notebook are based on version 1.7 of *spreg*.  To avoid some arguably obnoxious new features of *numpy* 2.0, it is necessary to include the `set_printoptions` command if you are using a Python 3.12 environment with numpy 2.0 or greater.

In [4]:
import numpy as np
import spreg
np.set_printoptions(legacy="1.25")

### Functions Used

- from geopandas:
  - astype
  
- from libpysal.weights:
  - Queen.from_dataframe
  - transform

- from spreg
  - OLS

### Variable Names

OLS estimation is illustrated in the context of the *immigrant paradox*. This is based on
four variables from the SDOH data set: **YPLL_rate** (an index measuring premature mortality, i.e., higher values are worse health outcomes), **HIS_ct** (economic hardship index), **Blk14P** (percent Black population), and **Hisp14P** (percent Hispanic population).

The easiest way to specify a generic regression model is to first create lists with the variable names for the dependent variable (**y_name**), the explanatory variables (**x_names1** and **x_names2**), and the data set name (optional, as **ds_name**). In this way, all the regression commands pertain to these generic variable names and do not need to be adjusted for different specifications and/or data sets.

The variables can also be loaded directly from the *pandas* or *geopandas* data frame. The setup here uses the variable names specified in the respective **y_name**, **x_names**, etc. lists. 

Note that there is no constant term in the x matrices. A constant vector is included by default in the `spreg.OLS` routine.

In [5]:
y_name = 'YPLL_rate'
x_names1 = ['Blk14P','Hisp14P']
x_names2 = ['Blk14P','Hisp14P','HIS_ct']
ds_name = 'Chi-SDOH'

y = dfs[y_name]
x1 = dfs[x_names1]
x2 = dfs[x_names2]

### Spatial Weights

As previously discussed, spatial weights are an essential part of any spatial autocorrelation analysis and spatial regression. To illustrate the specification tests for spatial effects, we use a queen contiguity weights matrix constructed from the geodata frame, as `libpysal.weights.Queen.from_dataframe` with as arguments the geodata frame and optionally the `ids` (recommended). For the Chicago data, the ID variable is **OJECTID**. To make sure the latter is an integer (it is not in the original data frame), its type is changed by means of the `astype` method.

As created, the weights are simply 1.0 for binary weights. To turn the weights into row-standardized form, a *transformation* is needed, `wq1.transform = 'r'`. We also specify a name for the weights to be included in the output listing.

In [6]:
dfs = dfs.astype({'OBJECTID':'int'})
wq1 = libpysal.weights.Queen.from_dataframe(dfs,ids='OBJECTID')
wq1.transform = 'r'
w_name = 'Chi-SDOH_q'
wq1

### OLS Regression

A first illustration uses the simplest of OLS regressions, where only y and X are specified as arguments in the `spreg.OLS` command. 

The resulting OLS object has many attributes. An easy (the easiest) way to list the results is to print the `summary` attribute.

First, the regression with the two ethnicity explanatory variables is considered. The dependent variable is **y** and the explanatory variables are contained in **x1**.

The default setting for OLS regression is to always include the non-spatial diagnostics, with `nonspat_diag=True`. Since this is the default, this argument does not have to be set. The result is a summary output that contains various measures of fit, the coefficient estimates, as well as diagnostics for multicollinearity, non-normality and heteroskedasticity. These are considered more closely below.

In [7]:
ols1 = spreg.OLS(y, x1, name_ds=ds_name)
print(ols1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :        None
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           3
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         788
R-squared           :      0.6023
Adjusted R-squared  :      0.6013
Sum squared residual: 3.13956e+09                F-statistic           :    596.8169
Sigma-square        : 3984214.804                Prob(F-statistic)     :    1.6e-158
S.E. of regression  :    1996.050                Log likelihood        :   -7131.628
Sigma-square ML     : 3969104.002                Akaike info criterion :   14269.255
S.E of regression ML:   1992.2610                Schwarz criterion     :   14283.275

------------------------------------------------------------

#### Immigrant paradox

As it turns out, the initial regression result is somewhat misleading, which is revealed when the economic hardship variable is included. This is implemented by specifying **x2** as the x-variable argument.

In [8]:
ols2 = spreg.OLS(y,x2,name_ds=ds_name)
print(ols2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :        None
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           4
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         787
R-squared           :      0.6330
Adjusted R-squared  :      0.6316
Sum squared residual: 2.89733e+09                F-statistic           :    452.5271
Sigma-square        : 3681490.057                Prob(F-statistic)     :  8.593e-171
S.E. of regression  :    1918.721                Log likelihood        :   -7099.872
Sigma-square ML     : 3662873.167                Akaike info criterion :   14207.744
S.E of regression ML:   1913.8634                Schwarz criterion     :   14226.437

------------------------------------------------------------

The inclusion of economic hardship turned the coefficient of the Hispanic share from positive to
negative. This is the so-called *immigrant paradox*. All coefficients are highly significant, with an adjusted $R^2$ of 0.6316.

We now take a closer look at the various parts of the output.

The measures of fit on the left-hand column are all related to the sum of squared residuals. This is listed, as well as two measures of $\sigma^2$ (one from division by the degrees of freedom, the other - ML - from division by the number of observations) and their square roots, the S.E. of regression.

On the right-hand side are the results of an F-statistic for the joint significance of the coefficients and its associated p-value, the log-likelihood $L$ (under the assumption of normality) for use in comparisions with spatial models, and two adjustments of the log-likelihood for the number of variables in the model, the $AIC$ and $SC$, with $AIC = -2L + 2k$ and $SC = -2L + k.ln(n)$ ($SC$ is sometimes referred to as $BIC$). Whereas a better fit is reflected in a higher log-likelihood, it is the reverse for AIC and SC (lower is better).

Below the listing of coefficients are the multicollinearity condition number, the Jarque-Bera test on normality of the errors and two tests for heteroskedasticity (random coefficients): the Breusch-Pagan LM test and the more robust (against non-normality) Koenker-Bassett test.

There is no evidence of a problem with multicollinearity (typically associated with values larger than 30), but a strong indication of both non-normality and heteroskedasticity.

#### Variance Inflation Factor (VIF)

The Variance Inflation Factor or VIF is an alternative to the multicollinearity condition number to assess the sensitivity of the regression results to the presence of multicollinearity. The condition number is a measure computed for all variables jointly and assesses the degree of linear dependence among all the columns of $X$. Instead, the VIF is computed for each variable separately. 

The VIF depends on the fit of a regression of the variable in question on all other x-variables, e.g., $R^2_k$ for $x_k$. $1 - R^2_k$ is called the *tolerance* (where $R^2_k$ is the unadjusted $R^2$). The more collinear $x_k$ is with the other variables, the larger $R^2_k$ and thus the lower the tolerance. The VIF is then the inverse of the tolerance.

The VIF is not reported as part of the standard regression output, since it requires considerable additional computation, but it is available by setting the argument `vif = True` (the default setting is `vif = False`). The output is augmented with a listing of the VIF and corresponding tolerance (its inverse) for each coefficient.

While there is no indication of a multicollinearity problem in the current set of results, it may still be informative to check which variables are the most correlated with the others.

In [9]:
ols2b = spreg.OLS(y, x2, vif=True, name_ds=ds_name)
print(ols2b.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :        None
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           4
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         787
R-squared           :      0.6330
Adjusted R-squared  :      0.6316
Sum squared residual: 2.89733e+09                F-statistic           :    452.5271
Sigma-square        : 3681490.057                Prob(F-statistic)     :  8.593e-171
S.E. of regression  :    1918.721                Log likelihood        :   -7099.872
Sigma-square ML     : 3662873.167                Akaike info criterion :   14207.744
S.E of regression ML:   1913.8634                Schwarz criterion     :   14226.437

------------------------------------------------------------

The output is the same as before, except that now the VIF measures are included at the bottom. In this result, **Blk14P** has the highest VIF at 4.323. However, this is not very high. When there are more serious multicollinearity problems, this value can be much higher. For example, with $R^2_k = 0.95$, the VIF would be 20, and with $R^2_k = 0.99$, it would be 100.

In the example, the corresponding tolerance is 0.231. In other words, a regression of **Blk4P** on the other two variables would have an (unadjusted) $R^2$ of 0.769. This can be readily verified in a simple regression by specifying the respective subsets of the **dfs** data frame. We set `nonspat_diag = False` to avoid the additional items in the output listing.

In [10]:
regv = spreg.OLS(dfs["Blk14P"],dfs[["Hisp14P","HIS_ct"]],nonspat_diag = False)
print(regv.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :      Blk14P                Number of Observations:         791
Mean dependent var  :     37.2082                Number of Variables   :           3
S.D. dependent var  :     40.6568                Degrees of Freedom    :         788
R-squared           :      0.7687
Adjusted R-squared  :      0.7681

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     t-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       -18.51830         2.12155        -8.72868         0.00000
             Hisp14P        -1.06303         0.02483       -42.81045         0.00000
              HIS_ct         2.09558         0.05289        

The unadjusted $R^2$ in this regression is indeed 0.769.

#### Other Options

Other options for the `OLS` call are the inclusion of a White test against heteroskedasticity, as `white=True`, and the computation of robust standard errors. Two forms of the latter are supported: `robust = "white"` for heteroskedastic-robust standard errors, and `robust = "hac"` for heteroskedastic *and* spatial correlation robust standard errors. The latter also require the specification of a kernel weights matrix.

Finally, it is now also possible to obtain the regression coefficient estimates, their standard errors, t-values and p-values as a latex-formatted table, by setting the argument `latex = True` (the default is `False`).

### Diagnostics for Spatial Effects

In this section, the basic regression diagnostics for spatial autocorrelation are introduced. These include the classic Lagrange Multiplier/Rao Score tests for lag and error dependence developed during the 1980s and 1990s. In addition, the recent tests for the Spatial Durbin specification developed by Koley and Bera (2024) are covered as well. The tests are detailed in Anselin and Rey (2014), *Modern Spatial Econometrics in Practice* and in Anselin, Serenini and Amaral (2024). *Spatial Econometric Model Specification Search: Another Look* (DOI: 10.13140/RG.2.2.10650.86721), as well as in the references therein.

To obtain the spatial diagnostics, the argument `spat_diag = True` must be set, with, in addition, `moran = True` if Moran's I is desired. Also, a spatial weights matrix must be specified as the `w` argument (optionally with its name in `name_w`).

#### Two explanatory variables

First, this is illustrated for the immigrant paradox regression with just two explanatory variables, i.e., with the arguments **y** and **x1**. The call to `spreg.OLS` is the same as before, but now with the arguments for the spatial diagnostics included.

In [11]:
ols1b = spreg.OLS(y, x1, w=wq1, spat_diag=True, moran = True,
                  name_w=w_name, name_ds=ds_name)
print(ols1b.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           3
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         788
R-squared           :      0.6023
Adjusted R-squared  :      0.6013
Sum squared residual: 3.13956e+09                F-statistic           :    596.8169
Sigma-square        : 3984214.804                Prob(F-statistic)     :    1.6e-158
S.E. of regression  :    1996.050                Log likelihood        :   -7131.628
Sigma-square ML     : 3969104.002                Akaike info criterion :   14269.255
S.E of regression ML:   1992.2610                Schwarz criterion     :   14283.275

------------------------------------------------------------

The regression output is augmented with a section at the bottom, entitled DIAGNOSTICS FOR SPATIAL DEPENDENCE. It is organized into two parts, one dealing the SAR-Error model as the alternative specification, the other with the Spatial Durbin specification (SDM). The *robust* LM tests are only robust to one of the other two possible alternatives. For example, the robust LM-Lag test in the SAR-Error part is robust to the presence of error autocorrelation, but not to a possible misspecification in the form of SLX. In contrast, the robust LM-Lag test in the Spatial Durbin part is robust to the presence of an SLX term, but not to spatial error autocorrelation. As Koley and Bera (2023) have shown, it is not possible to develop a test statistic that is robust to both other forms of misspecification, due to the lack of identification of the parameters in the GNS model. They refer to this as the *two out of three* problem.

The SAR-Error part shows Moran's I, the LM tests and their associated robust forms, first for Lag and then for Error. The standardized z-value for Moran's I is 6.521 and highly significant.

The LM-Lag test (26.211) is highly significant, but its robust form (0.258) is not. In contrast, both LM-Error (39.569) and the associated robust form (13.615) are highly significant, suggesting an Error alternative. Finally, the joint test for Lag and Error is highly significant as well. This test is not always indicative of the need for a higher order alternative, since it has high power against the single alternatives as well.

The upshot of these statistics is strong evidence towards a spatial error alternative.

The final set of tests are the LM tests in a Spatial Durbin context. The LM test for the coefficients of WX is given first, with its robust counterpart. The degrees of freedom (DF, 2) match the number of explanatory variables (not counting the constant term). It this example, the LM test of 0.836 is not significant, but its robust counterpart of 14.193 is (robust to the presence of a spatial lag). The LM test for Lag has the same value as in the SAR-Error context (26.211), but its robust form (39.569) is now robust to the presence of an SLX term and is highly significant. The joint test for Spatial Durbin (40.404) has degrees of freedom equal to the number of explanatory variables + 1, or 3 in this case. It is highly significant.

Koley and Bera (2024) suggest the following interpretation of these results. First, consider whether the joint test is significant, which it clearly is. Then consider each of the robust forms of the test statistic, which are also both highly significant in this example. This would point to a Spatial Durbin alternative. However, there may be some other misspecifications going on here, since the values of the robust tests are *larger* than their original counterparts, which is not standard behavior. In a strict sense, the robust tests are robust to *small* departures from the null and they should be *smaller* than the original test, since they *correct* for the ignored misspecification. When this inequality does not hold, there is an indication that the misspecification is more major than the correction is able to accommodate.

Clearly, the recommendations given by the SAR-Error and the Spatial Durbin contexts are at odds. 

#### Three explanatory variables

With the hardship indicator (**HIS_ct**) included in the regression, the call uses **x2**, but is otherwise identical to the previous one.

In [12]:
ols2c = spreg.OLS(y, x2, w=wq1, spat_diag=True, moran = True,
                  name_w=w_name, name_ds=ds_name)
print(ols2c.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           4
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         787
R-squared           :      0.6330
Adjusted R-squared  :      0.6316
Sum squared residual: 2.89733e+09                F-statistic           :    452.5271
Sigma-square        : 3681490.057                Prob(F-statistic)     :  8.593e-171
S.E. of regression  :    1918.721                Log likelihood        :   -7099.872
Sigma-square ML     : 3662873.167                Akaike info criterion :   14207.744
S.E of regression ML:   1913.8634                Schwarz criterion     :   14226.437

------------------------------------------------------------

The inclusion of the hardship indication in the regression specification changes not only the interpretation of the regression coefficients, but also greatly affects the results for the spatial diagnostics. In the SAR-Error context, Moran's I is still significant, but much less so than before. The LM-Lag statistic is also still significant, but its robust form is not. There is now only the weakest of evidence for the Error case (5.705 with a p-value of 0.02). Neither of the robust tests are significant. The joint LM test (7.329) only achieves a p-value of 0.0256, which provides very weak evidence (and not significant for p=0.01). 

The situation is completely different on the Spatial Durbin front. Now, the values for the robust forms of the tests are smaller than the original counterparts, as they should be. There is strong evidence for an SLX alternative. Following the Koley-Bera (2024) recommendations, interest focuses on the robust forms of the one-directional tests, since the joint test is highly significant. The robust Lag test (5.705) is no longer significant for a p-value of 0.01 (p=0169), whereas the robust WX test (17.742) is still highly significant (p=0.0005). This would suggest an SLX alternative, or possibly, with a lower standard for significance (e.g., p=0.05), a Spatial Durbin model.

## 2. Specification Search

In this section, we review five different strategies for a spatial regression model specification search, following the principles outlined in Anselin, Serenini and Amaral (2024) *Spatial Econometric Model Specification Search: Another Look* (DOI: 10.13140/RG.2.2.10650.86721). Three strategies are forward search strategies, from specific to general (STGE), and two are backward search strategies, from general to specific (GETS). They are based on a range of Lagrange Multiplier test statistics as well as simple asymptotic t-tests on the significance of the spatial parameters.

### Additional Functionality Used

- from spreg:
  - spsearch.stge_classic
  - spsearch.stge_kb
  - spsearch.stge_pre
  - spsearch.gets_gns
  - spsearch.gets_sdm

We proceed with the same dependent variable as before, contained in **y**, as well as the three explanatory variables in **x2**. We also use queen contiguity as the spatial weights, contained in **wq1**.

The results for the baseline regression, with a range of diagnostics for spatial autocorrelation, are given in the listing for **ols2c** above.

### STGE-Classic

The first forward strategy, the Classic Specific-to-General (STGE-Classic), is based on the application of the Lagrange Multiplier tests for
the lag and error alternatives and their robust counterparts.

The logic is a slight adjustment to the flow chart presented in Figure 5.1 on p. 110 of Anselin and Rey (2014). It is augmented with the AK test for
remaining spatial autocorrelation in a spatial lag model. 

The specification search is based on the results of a non-spatial
OLS regression and consists of the following steps:
- compute $LM_{\rho}$ and $LM_{\lambda}$: if none are significant, stick with the classic non-spatial regression (for simplicity, hereafter referred to as OLS); if one is significant and the other is not, select that alternative (Lag
or Error); if both are significant, move to the next step
- compute the robust versions $LM_{\rho}^*$ and $LM_{\lambda}^*$: if one is significant and the
other is not, go with that alternative; if both are significant, move to the next step
- estimate a spatial lag model (i.e., including $Wy$) by means of Spatial 2SLS and compute the AK test for remaining error autocorrelation: if the latter is significant, then the alternative is SAR-Error, otherwise it is Lag.

The last step is an addition to the original flow chart. As it turns out, in empirical practice, the situation where both robust tests remain significant occurs quite frequently (especially with larger data sets), which
leaves the orginal specification search unresolved. The AK test is preferred over the joint $LM_{\rho\lambda}$ test since the latter tends to over-reject the null, even when only one spatial parameter is significant.

Importantly, the STGE-Classic logic does not consider the spatial Durbin model as an alternative, and thus
it cannot detect SLX, SDM, SLX-Error or GNS.

This strategy is invoked as `spsearch.stge_classic`. The return value is a tuple with the final model and a regression object for the final model. Three required arguments are `y`, `x` and `w`. The other general optional arguments are the same as for a standard regression, e.g., `robust`, `w_lags` and `sig2n_k`.

Three important special optional arguments are `p_value`, set to 0.01 as the default, `finmod`, a flag for the estimation of the final model (default `True`), and `mprint`, a flag for listing the output summary of the final model (default `True`). With the default values for these arguments, the call yields the final model selected and the summary output.

In [13]:
mod, finreg = spreg.spsearch.stge_classic(y,x2,w=wq1,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-Classic: LAG
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
      

The model selected is LAG, although the spatial autoregressive coefficient is only marginally significant. The estimation of this model and the interpretation of the coefficients and impact measures are considered in Sections 5 and 6 of this notebook.

#### p-value

To assess the sensitivity of the result to the p-value used as the decision criterion, `p_value` is set to 0.05.

In [14]:
mod, finreg = spreg.spsearch.stge_classic(y,x2,w=wq1,name_ds=ds_name,
                      p_value=0.05)

Model selected by STGE-Classic: LAG_Nr
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :       False
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
   

Again, the end result is a LAG specification, but it is achieved via the LAG_Nr route. This is when both initial LM tests are significant (LM-Error becomes significant for p=0.05), but neither of the robust ones are. Then the decision tree takes the least insignificant of the two, in this case LAG.

### STGE-KB

In Koley and Bera (2024), robust versions are derived for the LM tests for $\rho$ and $\gamma$ in the spatial Durbin model. Again, the point of departure is
an OLS regression of the classic non-spatial specification. They recommend the following strategy:
- compute the joint $LM_{\rho\gamma}$ test for SDM: if the test is not significant, stick with the
       non-spatial specification; if the null is rejected, move to the next step
- compute the robust $LM_{\rho}^*$ and $LM_{\gamma}^*$ tests: if one is significant and the other is not, select
that alternative; if both are significant, select SDM.

Given the two out of three problem, these LM tests do not consider the SAR-Error model in the decision tree. As a result, the strategy cannot identify the Error, SAR-Error, SLX-Error and GNS alternatives.

This second strategy is invoked with `spsearch.stge_kb`. The required arguments and options are the same as for `stge_classic`.

In [15]:
mod, finreg = spreg.spsearch.stge_kb(y,x2,w=wq1,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-KB: SLX
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           7
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         784
R-squared           :      0.6420
Adjusted R-squared  :      0.6392
Sum squared residual: 2.82674e+09                F-statistic           :    234.2927
Sigma-square        : 3605541.065                Prob(F-statistic)     :  4.384e-171
S.E. of regression  :    1898.826                Log likelihood        :   -7090.117
Sigma-square ML     : 3573633.622                Akaike info criterion :   14194.234
S.E of regression ML:   1890.4057                Schwar

From the OLS diagnostics for Spatial Durbin in **ols2c**, the joint LM test for SDM is highly significant. The next step is then to check the robust LM tests for WX and Lag. Of these, only the test for WX is significant at p=0.01. As a result, the model selected is the SLX model. The diagnostics for remaining spatial effects are not significant at p=0.01, further confirming the choice of SLX as the final model.

### STGE-Pre

Neither the STGE-Classic nor the STGE-KB strategies consider the full array of spatial
model alternatives. A third forward strategy accomplishes this by means of a pre-test. The starting point
is again an OLS estimation of the classic non-spatial regression model. Based on the Koley-Bera $LM_{\gamma}$
and $LM_{\gamma}^*$ tests on the SLX parameters, a decision is made whether or not to include
$WX$ in the estimation. If the null is not rejected, the initial estimates are used and the STGE-Classic strategy is followed. If the null is rejected, the model is re-estimated as an SLX specification and the
STGE-Classic strategy is applied to these results (an SLX specification is also estimated by OLS, so the STGE-Classic strategy is appropriate).

This yields the full array of alternatives:
- from the original regression: OLS, Lag, Error and SAR-Error
- from the SLX estimation: SLX, SDM, SLX-Error and GNS.

The STGE-Pre strategy is invoked with `spsearch.stge_pre`. The required arguments and options are again the same.

In [16]:
mod, finreg = spreg.spsearch.stge_pre(y,x2,w=wq1,
                        name_ds=ds_name,name_w=w_name)

Model selected by STGE-Pre: SLX
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           7
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         784
R-squared           :      0.6420
Adjusted R-squared  :      0.6392
Sum squared residual: 2.82674e+09                F-statistic           :    234.2927
Sigma-square        : 3605541.065                Prob(F-statistic)     :  4.384e-171
S.E. of regression  :    1898.826                Log likelihood        :   -7090.117
Sigma-square ML     : 3573633.622                Akaike info criterion :   14194.234
S.E of regression ML:   1890.4057                Schwa

Starting with the LM results for Spatial Durbin in the OLS regression (**ols2c**), both the LM test for WX and its robust form are significant at p=0.01. Therefore, the next step is the SLX regression. As pointed out above, the diagnostics in the SLX regression obtained for p=0.01 with STGE-KB show that none of the LM tests are significant at p=0.01. Therefore, the selected model is SLX.

### GETS-GNS

The point of departure in the GETS-GNS strategy is the estimation of a full GNS model. Next, the significance of the individual coefficients is assessed. When one or more coefficients are not significant,
the simplified model (with zero restrictions imposed) is estimated and the process is repeated. All estimations
use IV-GMM methods.

One additional step is carried out when the selected model turns out to be SDM. In that case, the common
factor hypothesis is tested. If the latter is not rejected, then the selected model is the Error model rather than
the unrestricted SDM.

This strategy is included in `spsearch` only for the sake of completeness. As the simulation results in Anselin, Serenini and Amaral (2024) show, it is generally unreliable and very often yields parameter estimates outside the acceptable range. In `spreg`, the parameter space is enforced with the `hard_bound = True` option (See Section 5). This option cannot be relaxed here since a model that yields unacceptable parameter estimates should not be considered a valid specification.

The GETS-GNS strategy is invoked with `spsearch.gets_gns`. The required arguments and options are again the same.

In [17]:
mod, finreg = spreg.spsearch.gets_gns(y,x2,w=wq1,
                        name_ds=ds_name,name_w=w_name)

Exception: GNS parameters out of bounds


In this example, the GNS model is not a valid point of departure. As mentioned, this strategy is generally unreliable and not recommended for empirical practice.

### GETS-SDM

The GETS-SDM strategy is a hybrid that contains aspects of both General-to-Specific (GETS) and Specific-to-General (STGE) approaches. In contrast to GETS-GNS, the point of
departure is not the GNS specification, but instead an SDM model is used. If both $\rho$ and $\gamma$ coefficients are significant, then
a common factor test is carried to assess whether the alternative may be an Error model. If the common factor null hypothesis cannot
be rejected, then the model is Error. In the other case, the SDM specification is kept and
an AK test is carried out on remaining error autocorrelation. If the
null is rejected, then the alternative is GNS, if not, then it remains SDM.

If neither of the coefficients in the SDM model are significant, then the LM-Error tests are used in an OLS regression
of the model without $Wy$ and $WX$ terms, allowing both OLS and Error alternatives. If $\rho$ is significant and $\gamma$ is not,
then an AK test is carried out, yielding either SAR-Error or Lag as the alternatives. In the reverse case, with $\gamma$ significant
and $\rho$ not, then the LM Error tests are used to choose between the SLX and SLX-Error alternatives.

This approach allows the full array of spatial alternatives to be considered.

The GETS-SDM strategy is invoked with `spsearch.gets_sdm`. The required arguments and options are again the same.

In [18]:
mod, finreg = spreg.spsearch.gets_sdm(y,x2,w=wq1,
                        name_ds=ds_name,name_w=w_name)

Model selected by GETS-SDM: LAG
REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :   YPLL_rate                Number of Observations:         791
Mean dependent var  :   5017.1325                Number of Variables   :           5
S.D. dependent var  :   3161.3278                Degrees of Freedom    :         786
Pseudo R-squared    :      0.6374
Spatial Pseudo R-squared:  0.6334

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT       676.18274       253.42997         2.66812         0.00763
              Blk14P        34.93664         5.01195         6.97067         0.00000
          

The point of departure is the SDM model, which is shown in the output of the STGE-Pre strategy. The next step is based on the least significant spatial parameter. If that is the spatial autoregressive coefficient, then the next step is SLX. However, in the current example, two of the WX parameters are less significant than the spatial autoregressive coefficient, and in that case the next step is the LAG model. The motivation is that the consequences of ignoring a lag term are more severe than ignoring an SLX specification. The AK test in the lag model does not reveal remaining autocorrelation, so the recommendation is the LAG specification. 

### Overall Assessment

The results of the simulation results in Anselin, Serenini and Amaral (2024) point to the crucial role played by the $WX$ terms. When only classic models are considered (specifically,
no spatial Durbin), the augmented STGE-Classic strategy is superior to all and provides reliable guidance as to the proper alternative. 
In contrast, when $WX$ is part of the true DGP, this is no longer the case. Both STGE-Classic, which does not consider SDM as a DGP, and 
STGE-KB, which does, but ignores spatial errors, fall short. Of the three strategies that take into account SDM, STGE-Pre is clearly superior. In addition, it also
performs well in the classic setting, although not quite as well as STGE-Classic.

GETS-SDM
does well in some settings, but not reliably. GETS-GNS yields unreliable results and should be avoided as a strategy. 

While these search strategies provide an easy way to asses which spatial model may be relevant in a given empirical situation, nothing is a replacement of an explicit and careful model specification based on substantive considerations. Also, other misspecifications, such as endogeneity, heteroskedasticity and nonlinearity may affect the indications provided by the test diagnostics.

## 4. Estimating SLX Models

In this section, a closer look is taken at the estimation of SLX models in their traditional linear specification, i.e., using $WX$ as the spatially lagged explanatory variables, where $W$ is a spatial weights matrix.

The `spreg` package implements the inclusion of spatially lagged explanatory variables in any specification by means of a non-zero `slx_lags` argument. This allows for the estimation of non-spatial linear models by means of OLS, as well as spatial Durbin and SLX-Error models by means of specialized methods. In addition, the `slx_vars` argument allows for a selective inclusion of $WX$ terms. This is useful when some coefficient estimates have the wrong sign or a magnitude that runs counter to the distance decay implied by Tobler's law. The spatially lagged explanatory variables can be created using both classic row-standardized weights, as well as kernel weights. The latter introduce of form of nonlinearity in the spatial averaging.

As of Version 1.7, the `NSLX` module introduces the estimation of nonlinear SLX models, such as a negative exponential distance function and an inverse distance power transformation. The `NSLX` module is still somewhat experimental and under active development, but the current version is stable. Because of time constraints, it is not considered in this workshop.

### Classic Weights

For this exercise, a different model specification is used. It is purely to illustrate the various estimation methods. It relates the variable **HIS_ct** (economic hardship index) to **Blk14P** (percentage Black households), **Hisp14P** (percentage Hispanic households), and **EP_NOHSDP** (percentage households without high school education).

In [19]:
y2 = dfs['HIS_ct'] 
x3 = dfs[['Blk14P','Hisp14P','EP_NOHSDP']]

The linear SLX model with queen contiguity weights is invoked by means of `OLS` with the usual arguments, and with `slx_lags=1`. Spatial diagnostics are included as well (`spat_diag=True`). The same queen contiguity weights are used as before.

In [20]:
slxw1 = spreg.OLS(y2, x3, w=wq1, slx_lags=1, spat_diag=True, 
                  name_w=w_name, name_ds=ds_name)
print(slxw1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :      HIS_ct                Number of Observations:         791
Mean dependent var  :     39.7301                Number of Variables   :           7
S.D. dependent var  :     13.8098                Degrees of Freedom    :         784
R-squared           :      0.8442
Adjusted R-squared  :      0.8430
Sum squared residual:     23475.4                F-statistic           :    707.9250
Sigma-square        :      29.943                Prob(F-statistic)     :  1.766e-312
S.E. of regression  :       5.472                Log likelihood        :   -2463.288
Sigma-square ML     :      29.678                Akaike info criterion :    4940.576
S.E of regression ML:      5.4478                Schwarz criterion     :    4973.289



The overall fit of the model yields an adjusted $R^2$ of 0.84, with a sum of squared residuals of 23,475.4. All lag terms are highly significant, but two of them are negative (**W_Blk14P** and **W_Hisp14P**), while the third one (**W_EP_NOHSDP**) is positive. The interpretation of negative coefficients for the spatial lags is difficult, so this may point to a misspecification. While significant, the coefficient of **W_EP_NOHSDP** is slightly larger than that of the coefficient itself, which runs counter to Tobler's law.

The inclusion of the SLX terms affects the spatial diagnostics. In the curent example, the original LM-Lag and LM-Error tests are very significant, but their robust forms are not.

Since the coefficient of **W_EP_NOHSDP** was least problematic, `slx_vars` is used to remove the other lag terms. To this effect, it is set to the list `[False,False,True]` (the default is otherwise `slx_vars="All"`), which will eliminate **W_Blk14P** and **W_Hisp14P** from the regression specification. All the other arguments are as before.

In [21]:
slxw2 = spreg.OLS(y2, x3, w=wq1, slx_lags=1, slx_vars=[False,False,True], 
                  spat_diag=True, name_w=w_name, name_ds=ds_name)
print(slxw2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :      HIS_ct                Number of Observations:         791
Mean dependent var  :     39.7301                Number of Variables   :           5
S.D. dependent var  :     13.8098                Degrees of Freedom    :         786
R-squared           :      0.8399
Adjusted R-squared  :      0.8391
Sum squared residual:     24117.4                F-statistic           :   1031.0245
Sigma-square        :      30.684                Prob(F-statistic)     :  6.646e-311
S.E. of regression  :       5.539                Log likelihood        :   -2473.959
Sigma-square ML     :      30.490                Akaike info criterion :    4957.919
S.E of regression ML:      5.5218                Schwarz criterion     :    4981.285



The inclusion of the spatial lag for just **EP_NOHSDP** makes the coefficient of **Hisp14P** not significant. On the other hand, the coefficient of the lag term is now smaller than that of the non-lagged variable, conforming to Tobler's law. The fit still gives an adjusted $R^2$ of 0.84, with a residual sum of squares of 24,117.4.

The spatial diagnostics seem to point to potential spatial error dependence (i.e., an SLX-Error specification), with both the LM test and its robust form highly significant.

### Kernel Weights

As mentioned, kernel weights can be used to compute the $WX$ terms in the SLX model. In this example, we will use triangular weights computed using the `libpysal.weights.Kernel` function, with a matrix of x-y coordinates passed. The X and Y coordinates contained in the geodataframe **dfs** are used as `COORD_X` and `COORD_Y`. 

First, the respective columns from the data frame are turned into a numpy array. Next, 
`libpysal.weights.Kernel` takes the array as the first argument, followed by a number of options. To have a variable bandwidth that follows the 10 nearest neighbors, 
`fixed = False` (the default is a fixed bandwidth) and `k=10`. The kernel function is selected as `function="triangular"` (this is also the default, but it is included here for clarity). For convenience, the diagonal elements are set to the value of one (achieved by means
of `diagonal=True`), even though in the computation of $WX$, these diagonals are set to zero. When `diagonal=False`, the actual value from the kernel function is entered on the diagonal, but this is immaterial for the kernel-SLX estimation.

In [22]:
coords = np.array(dfs[['COORD_X','COORD_Y']])
kwtri10 = libpysal.weights.Kernel(coords,fixed=False,k=10,
                                   function="triangular",diagonal=True)
kwtri10_name = 'Chi-SDOH_tri10'
kwtri10

Kernel weights can be passed to an estimation routine to compute the SLX terms in the same way as other weights, but only for `slx_lags = 1`. If a higher order lag is specified, `spreg` will raise an `Exception`. Also, the interpretation of the results is different from the standard linear case. While the diagonal elements as set to zero, just as for other spatial weights, kernel weights are *not* row-standardized. As a result, the magnitude of the associated coefficients is not directly comparable to the non-lagged counterparts.

Also, importantly, the results of the diagnostics for spatial effects are not valid for kernel weights without adjustments, so they cannot be activated.

In all other respects, the command is identical to the standard case.

In [23]:
slxk1 = spreg.OLS(y2, x3, w=kwtri10, slx_lags=1, 
                  name_w=kwtri10_name, name_ds=ds_name)
print(slxk1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :Chi-SDOH_tri10
Dependent Variable  :      HIS_ct                Number of Observations:         791
Mean dependent var  :     39.7301                Number of Variables   :           7
S.D. dependent var  :     13.8098                Degrees of Freedom    :         784
R-squared           :      0.8420
Adjusted R-squared  :      0.8407
Sum squared residual:     23811.7                F-statistic           :    696.0807
Sigma-square        :      30.372                Prob(F-statistic)     :  4.638e-310
S.E. of regression  :       5.511                Log likelihood        :   -2468.914
Sigma-square ML     :      30.103                Akaike info criterion :    4951.828
S.E of regression ML:      5.4866                Schwarz criterion     :    4984.541

As in the linear case, all lag coefficient are significant, with negative signs for **W_Blk14P** and **W_Hisp14P**. In contrast to the linear case, the magnitude of the coefficients of the lag terms is not directly comparable to the original coefficient.

In terms of fit, the results are similar to that of the linear model although slightly worse, with an adjusted $R^2$ around 0.84 and a sum of squared residuals of 23,811.7 (compared to 23,475.4 for queen contiguity).

## 5. Spatial Lag and Spatial Durbin models

This section deals with the estimation of the spatial Lag model and the Spatial Durbin model. Only instrumental variables estimation is considered. The maximum likelihood estimation in `spreg` is primarily included for pedagogical purposes and is therefore not treated here. 

The `spreg` module implements IV estimation of the spatial lag model in the `GM_Lag` class. The estimation of the Spatial Durbin model is implemented through the inclusion of the `slx_lags` argument. 

### Additional Functionality Used

- from spreg:
  - GM_Lag

To illustrate the estimation of these specifications, a descriptive model is used that relates the rate of uninsured households in a tract(for health insurance, **EP_UNINSUR**) to the lack of high school education (**EP_NOHSDP**), the economic deprivation index (**HIS_ct**), limited command of English (**EP_LIMENG**) and the lack of access to a vehicle (**EP_NOVEH**). Again, this specification is purely illustrative of a spatial lag model and does not have a particular theoretical or policy motivation.

The file names and variable names are set in the usual manner as **y3** and **x4**.

In [24]:
y3 = dfs['EP_UNINSUR']
x4 = dfs[['EP_NOHSDP','HIS_ct','EP_LIMENG','EP_NOVEH']]

### Baseline OLS Regression

Standard OLS and SLX regressions with spatial diagnostics are carried out to provide a point of reference. 

In [25]:
ols4 = spreg.OLS(y3, x4, w=wq1, spat_diag=True, 
                 name_w=w_name, name_ds=ds_name)
print(ols4.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :           5
S.D. dependent var  :      9.0543                Degrees of Freedom    :         786
R-squared           :      0.6358
Adjusted R-squared  :      0.6339
Sum squared residual:     23589.7                F-statistic           :    342.9769
Sigma-square        :      30.012                Prob(F-statistic)     :  1.061e-170
S.E. of regression  :       5.478                Log likelihood        :   -2465.209
Sigma-square ML     :      29.823                Akaike info criterion :    4940.419
S.E of regression ML:      5.4610                Schwarz criterion     :    4963.785

------------------------------------------------------------

The specification achieves an acceptable $R^2$ of about 0.63 and all coefficients are positive and highly significant.

The non-spatial diagnostics suggest non-normality as well as a hight degree of heteroskedasticity. There is no problem with multicollinearity.

The spatial diagnostics against the SARERR alternatives show very significant LM-Lag and LM-Error, but of the two robust tests, only RLM-Lag is highly significant (RLM-Error only at p < 0.03). Hence, there is a strong indication that a Lag rather than an Error alternative may be appropriate. While the joint LM test is also highly significant, this is likely due to a strong one-sided (Lag) alternative.

Interestingly, the diagnostics against a spatial Durbin alternative strongly support the latter. Both LM tests and their robust forms are highly significant, and so is the joint test. Moreover, the value for the robust forms of the test is smaller than the original, which is the expected behavior (although not always reflected in empirical practice).

In sum, in addition to a spatial Lag model as an alternative, the spatial Durbin specification deserves consideration as well.

As a baseline reference, OLS estimation of the SLX model is considered as well.

In [26]:
slx1 = spreg.OLS(y3, x4, w=wq1, slx_lags=1, spat_diag=True, 
                 name_w=w_name, name_ds=ds_name)
print(slx1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES WITH SPATIALLY LAGGED X (SLX)
-----------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :           9
S.D. dependent var  :      9.0543                Degrees of Freedom    :         782
R-squared           :      0.6559
Adjusted R-squared  :      0.6523
Sum squared residual:     22288.1                F-statistic           :    186.2889
Sigma-square        :      28.501                Prob(F-statistic)     :    2.1e-175
S.E. of regression  :       5.339                Log likelihood        :   -2442.761
Sigma-square ML     :      28.177                Akaike info criterion :    4903.521
S.E of regression ML:      5.3082                Schwarz criterion     :    4945.581



Relative to the classic regression model, the fit improves slightly, but the constant, **EP_NOHSDP** and **HIS_CT** become non-significant at p = 0.01 (they are marginally signifcant at p=0.05). All but one coefficient of the SLX terms are significant (**W_EP_NOVEH** is not). The signs and magnitudes of the SLX coefficients relative to their unlagged counterparts remain a bit confusing. Only for **EP_LIMENG** and **W_EP_LIMENG** are they the same, with the lag coefficient smaller than the unlagged one, in accordance with Tobler's law. The coefficient for **W_HIS_ct** is significant and larger than that of **HIS_ct**, while the latter is not significant at p = 0.01. In other words, the interpretation of these results in terms of distance decay and Tobler's law may be a bit problematic.

In terms of diagnostics, there is a slight problem with multicollinearity (often the case in SLX specifications), strong non-normality and evidence of heteroskedasticity. Both LM-tests are significant, but neither of the robust forms is. Based on the relative magnitudes of the test statistics, there is a slight indication of a possible Lag alternative, i.e., a spatial Durbin specification. However, this indication is not as strong as that provided by the LM-SDM test statistics in the classic specification.

### IV Estimation of the Lag Model

The endogeneity of the spatially lagged dependent variable can be addressed by means of instrumental variables (IV) estimation. This is implemented in the `spreg.GM_Lag` class. The estimation of the Spatial Durbin model is achieved through the inclusion of the `slx_lags` argument. 

The IV estimation of the spatial lag model is a customized implementation of the two stage least squares estimation, with the instruments for the spatially lagged dependent variable computed internally. The default setup requires the dependent variable, `y`, the explanatory variables (without a constant term), `x`, and the spatial weights `w`. The instruments are the spatially lagged explanatory variables, WX. They do not need to be specified separately. The order of spatial weights used as instruments is set by means of `w_lags` (the default is `w_lags = 1`).

Since the IV estimation does not guarantee that the spatial autoregressive coefficient obtained is within the acceptable parameter space, a `hard_bound = True` argument is set by default. This will generate an error message when the coefficient estimate is outside the parameter space. For `hard_bound = False`, the usual output listing will be provided, but some characteristics (such as the pseudo $R^2$ from the reduced form) cannot be computed for such coefficient estimates.

Also, for now, the `spat_impacts` argument is set to `None` since we will return to it in Section 6 of this notebook.

The AK-test for remaining residual spatial autocorrelation is included by default (i.e., `spat_diag = True`).

In [27]:
lag1 = spreg.GM_Lag(y3, x4, w=wq1, spat_impacts=None, 
                    name_w=w_name,name_ds=ds_name)
print(lag1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :           6
S.D. dependent var  :      9.0543                Degrees of Freedom    :         785
Pseudo R-squared    :      0.6840
Spatial Pseudo R-squared:  0.6474

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.41406         0.76860         0.53872         0.59008
           EP_NOHSDP         0.06480         0.03356         1.93098         0.05349
              HIS_ct         0.14529      

The spatial autoregressive coefficient of 0.377 is highly significant. Of the other coefficients, **EP_NOHSDP** becomes non-significant (as does the constant term), but the other coefficients remain highly significant. The coefficients of **HIS_ct**, and, to a lesser extent, **EP_LIMENG** become much smaller than their OLS counterparts, suggesting the latter may be biased. There is no evidence of remaining error autocorrelation provided by the AK test. Note how there are two pseudo $R^2$ values reported. The more reliable one is the *spatial* pseudo $R^2$, which is based on the reduced form of the spatial lag model.

### IV Estimation of Spatial Durbin Model

IV estimation of the Spatial Durbin model is a special case of `spreg.GM_Lag`, with the additional argument of `slx_lags=1` (or a larger value). Everything else remains the same. 

In the example, `spat_impacts` is again set to `None` for now, unlike the default. Another default setting is `spat_diag = True`, which yields the results for the both the AK test and the Common Factor Hypothesis test. To avoid these tests, `spat_diag` must be set to `False`. In the illustration, both arguments are listed explicitly for clarity.

In [28]:
spdur1 = spreg.GM_Lag(y3, x4, w=wq1, slx_lags=1,
                    name_w=w_name, name_ds=ds_name,
                    spat_impacts = None,
                    spat_diag = True)
print(spdur1.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :          10
S.D. dependent var  :      9.0543                Degrees of Freedom    :         781
Pseudo R-squared    :      0.6956
Spatial Pseudo R-squared:  0.6567

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -1.76763         1.10960        -1.59302         0.11115
           EP_NOHSDP         0.09563         0.03789         2.

Note how the list of **Instruments** now consists of the second order spatial lags, even though `w_lags = 1`. This is because the spatial Durbin model already includes the first order lags as explanatory variables. Using first order lags as instruments would result in perfect multicollinearity. This is detected internally in `GM_Lag`, so that the computation of the lagged instruments is adjusted accordingly.

The inclusion of the lagged explanatory variables has quite an effect, even though only **W_EP_NOHSDP** turns out to be significant of the four SLX terms. Also, the spatial autoregressive coefficient (0.509) is no longer significant. Three of the four lag coefficients (including the non-significant ones) have a negative sign, the opposite of the original regression coefficients. This raises the suspicion that the proper specification may be a spatial error model. The results of the Common Factory Hypothesis Test bear this out. At 4.083, it is *not* significant, which means that the common factor constraint can *not* be rejected. However, given the lack of significance of the spatial lag terms, this needs to be interpreted with caution. A better strategy would be to consider further refinements of the model specification by eliminating some lag terms by means of `slx_vars`, as in the SLX model. Specifically, since only **W_EP_NOHSDP** turned out to be significant, `slx_vars` is set to `[True, False, False, False]`

In [29]:
spdur2 = spreg.GM_Lag(y3, x4, w=wq1, slx_lags=1,
                    slx_vars = [True, False, False, False],
                    name_w=w_name, name_ds=ds_name,
                    spat_impacts = None,
                    spat_diag = True)
print(spdur2.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :           7
S.D. dependent var  :      9.0543                Degrees of Freedom    :         784
Pseudo R-squared    :      0.6931
Spatial Pseudo R-squared:  0.6494

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -0.89815         0.92502        -0.97096         0.33157
           EP_NOHSDP         0.10579         0.03700         2.

The results are again quite different. Now, the spatial autoregressive coefficient (0.614) is again significant, but the SLX term only marginally so (p=0.014). Other than the constant term, all the other main coefficients are significant again. Note how the instruments consist of the first order lags for three of the four explanatory variables, but the second order lag for the SLX term.

The AK-test provide some (weak) suggestion of remaining error autocorrelation.

## 6. Spatial Multipliers - Impacts

In models that include a spatially lagged dependent variable Wy, with or without additional spatially lagged explanatory variables, the
impact of a change in X on y is not simply the coefficient of X, as is the case in the standard regression model. Instead, the effect that results from changes in the neighboring values must also be accounted for. These are the *spatial multipliers*, *indirect effects* or *spatial impacts*.

In Kim, Phipps and Anselin (2003), it was shown that if the change in the explanatory variable is uniform across observations, the *spatial multiplier* is $1 / (1- \rho)$, with the total effect of a change in variable
$x_k$ amounting to $\beta_k / (1 - \rho)$. In the example above (**lag1**), the spatial multiplier in the spatial lag model would be 
1.0 / (1.0 - 0.37698) = 1.605.

The Kim et al. approach distinguishes between the direct effect, i.e., the coefficient of the $\beta$ coefficients as estimated, and the total effect, which corresponds to this coefficient times the multiplier. An indirect effect is then the difference between the two.

LeSage and Pace (2009) introduce a slightly different set of concepts and use the terms average direct impact ($ADI$), average indirect impact ($AII$) and average total impact ($ATI$) as summaries computed from the matrix expression $(I - \rho W)^{-1}$ in the reduced form, $(I - \rho W)^{-1}X\beta$. The main difference is that what they refer to as *direct* effect also includes some feedbacks as reflected in the diagonal elements of the inverse matris. As a result, in their approach, the direct effects will differ from the estimates for $\beta$. 

More formally, LeSage-Pace define $ADI$ as the average trace of the inverse matrix, or, $ADI = (1/n) tr[(I - \rho W)^{-1}] = (1/n) \sum_i (I - \rho W)^{-1}_{ii}$. The $ATI$ is the average of all the elements of the matrix, or, $ATI = (1/n) \sum_i \sum_j (I - \rho W)^{-1}$. Note that with some algebra, one can show that this equals $1 / (1 - \rho)$, the same as the total multiplier in the Kim et al. approach.

The $AII$ then follows as $ATI - ADI$. The actual impacts are obtained by multiplying the $\beta$ coefficient by respectively $ATI$, $ADI$ and $AII$.

The impact measures are listed in the spatial lag regression output when the `spat_impacts` argument is specified (it is by default set to `spat_impacts = "simple"`). Options include `simple` (Kim et al. approach), `full` and `power` (both based on LeSage-Pace, but with `full` using a dense matrix computation for the inverse, whereas `power` uses a power approximation with higher order weights), as well as `all`, for all three. In addition, any combination of methods can be passed in a list. For example, to obtain both Kim et al. and LeSage-Pace measures, the argument can be set as
`spat_impacts = ["simple","full"]`, as in the listing below. Since the default setting is `spat_impacts = "simple"`, when listing the impacts is not desired, `spat_impacts` must explicitly be set to `None`.

Based on extensive timing experiments for the LeSage-Pace approach, the `power` method is superior for data sets with more than 5,000 observations. For larger data sets, it quickly becomes the only viable option, being orders of magnitude faster than the brute force calculations. The Kim et al. approach has no such limitation, since it does not use any matrix calculations. 

Note that the reported impacts are only *average* effects.

### Impacts in a Spatial Lag Model

We now re-run the spatial lag specification with the `spat_impacts` argument active. The estimates are (of course) the same as before, the only difference is the addition of a Spatial Lag Model Impacts set of results at the bottom of the summary output listing.

In [30]:
lag2a = spreg.GM_Lag(y3, x4, w=wq1, 
                    spat_impacts=['simple','full'], 
                    name_w=w_name,name_ds=ds_name)
print(lag2a.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :           6
S.D. dependent var  :      9.0543                Degrees of Freedom    :         785
Pseudo R-squared    :      0.6840
Spatial Pseudo R-squared:  0.6474

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT         0.41406         0.76860         0.53872         0.59008
           EP_NOHSDP         0.06480         0.03356         1.93098         0.05349
              HIS_ct         0.14529      

The listing of **SPATIAL LAG MODEL IMPACTS** for the `simple` and `full` methods clearly illustrates the slight differences between the two approaches. The **Total** effect is the same for both, but the distribution of the effect between **Direct** and **Indirect** is slightly different. For the `simple` method, the **Direct** effects are identical to the coefficient estimates, but for the `full` method, they are slightly larger. This is due to the use of the diagonal elements of the inverse matrix, rather than the original estimates. As a result, some of the spillover feed-back effects are characterized as **Direct**. As a consequence, the part attributed to the **Indirect** effect is larger for the `simple` method than for the `full` method.

These measures should only be interpreted as rough indications of spatial spillovers since they are both based on a rather unrealistic assumption of a uniform change in the X-variable. Also, the **Total** effect is simply the original coefficient times the spatial multiplier. For example, for **EP_LIMENG**, this amounts to $0.3153 \times 1.605 = 0.5061$.

This total effect can be compared to that implied by the SLX model, which would be the sum of $\beta$ and $\gamma$. For example, for **EP_LIMENG**, this would be $0.38547 + 0.22040 = 0.60587$, which is actually larger than the total effect suggested by the spatial lag model. In part, this can be explained by recognizing that the SLX model is a truncated form of the reduced form for the spatial lag specification, i.e., only the first order contiguity elements from the reduced form are included. When ignoring the remainder, its impact tends to be attributed to the coefficients of $\beta$ and $\gamma$.

### Impacts in a Spatial Durbin Model

The impact measures for the Spatial Durbin model follow the same logic as in the spatial lag model. They are derived from the reduced form, which is now, ignoring the error term:
\begin{equation*}
y = (I - \rho W)^{-1} X \beta + (I - \rho W)^{-1} WX \gamma.
\end{equation*}
The effect of this is that the various multipliers must be applied to both $\beta$ and the matching $\gamma$ to compute the overall impacts.

As it turns out, the total multiplier is the same as in the lag model, i.e., $1.0 / (1.0 - \rho)$. However, to get the total effect, this factor needs to be multiplied by both $\beta$ and the matching $\gamma$.

As in the lag model, the total impact is the same for the Kim et al approach and the LeSage-Pace approach, i.e., $1.0 / (1.0 - \rho)$. In order to get the total effect, this factor needs to be multiplied by both $\beta$ and the matching $\gamma$. However, there is an important difference in the way the direct effect is computed.

In the Kim et al approach, the direct effect is the coefficient of $\beta$. This is a strict interpretation of direct effect, i.e., it considers the effect of $\gamma$ to be indirect. This is consistent with the interpretation of $\gamma$ in the SLX model, but it is not the approach used by LeSage-Pace in their original treatment. In their formulation, the $ADI$ is applied to both $\beta$ and $\gamma$ to compute the direct effect, typically yielding a larger value for the direct effect (as long as $\beta$ and $\gamma$ have the same sign). This approach is *not* followed by `spreg`. Instead, the direct effect is obtained by multiplying the $ADI$ with the $\beta$ coefficient only. As a result, the share attributed to the indirect impact will be larger than in original the LeSage-Pace formulation (in contrast, the latter is followed in the `R` `spatialreg` package).

In [31]:
spdur1a = spreg.GM_Lag(y3, x4, w=wq1, slx_lags=1,
                    name_w=w_name, name_ds=ds_name,
                    spat_impacts = ['simple','full'],
                    spat_diag = True)
print(spdur1a.summary)

REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES WITH SLX (SPATIAL DURBIN MODEL)
----------------------------------------------------------------------------------
Data set            :    Chi-SDOH
Weights matrix      :  Chi-SDOH_q
Dependent Variable  :  EP_UNINSUR                Number of Observations:         791
Mean dependent var  :     18.4752                Number of Variables   :          10
S.D. dependent var  :      9.0543                Degrees of Freedom    :         781
Pseudo R-squared    :      0.6956
Spatial Pseudo R-squared:  0.6567

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT        -1.76763         1.10960        -1.59302         0.11115
           EP_NOHSDP         0.09563         0.03789         2.

With $\rho = 0.50924$ as obtained here, the total multiplier follows as $2.038$. The total impact for the variable **EP_LIMENG** (ignoring for now that **W_LIMENG** turned out to be non-significant) would be $2.038 \times 0.37091 + 2.038 \times -0.04292 = 0.6683$, the value given in the **Total** column of the impacts listing. As in the lag model, the total impact is the same for the Kim et al approach and the LeSage-Pace approach. The main difference is in the way the direct effect is computed, which results in a different allocation between direct and indirect effects for the two approaches.

The total and direct/indirect impacts can be compared to those suggested by the simple spatial lag model and the SLX model. For example, for **EP_LIMENG**, the total impact of a uniform change in that variable was 0.606 in SLX, 0.5061 in the spatial lag model and 0.6683 in the spatial Durbin specification. Of this (using the Kim et al logic), 0.220 was spatial spillover (indirect effect) in SLX, 0.1908 in spatial lag, and 0.2974 in the spatial Durbin model.

As mentioned above, these impact measures are only summaries. A more meaningful indication of spatial spillovers would follow from a non-uniform change in (some of) the X variables through the use of the full reduced form. In addition, the average may mask some interesting spatial patterns. The `spreg.sputils` module contains the function `i_multipliers` that provides this detail.

## What we did not cover

Due to time constraints, it is impossible to cover the full range of functionality contained in the `spreg` package in this workshop. Some important aspects that we did not cover include the estimation of spatial error models (both ML and GMM), and the estimation of higher order models. The latter are implemented as special cases of `spreg.GMM_Error` (the generic spatial error estimation) by including a spatially lagged dependent variable through the argument `add_wy = True`. As mentioned in the introduction, there is also functionality to carry out spatial diagnostics in a probit model, and to implement various forms of spatial regimes regression. So far, panel data functionality is limited to spatial seemingly unrelated regression and spatial lag and error fixed effects. New functionality continues to be added.